In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv')
df.head()

,text,class
0,"plot : two teen couples go to a church party ,...",0
1,the happy bastard ' s quick movie review damn ...,0
2,it is movies like these that make a jaded movi...,0
3,""" quest for camelot "" is warner bros . ' first...",0
4,synopsis : a mentally unstable man undergoing ...,0


In [6]:
x = df.iloc[0, 0]

In [3]:
def tokenize(text):
    return text.split(" ")

In [19]:
import nltk

In [126]:
from nltk.corpus import stopwords
# nltk.download('stopwords')
stopwords = stopwords.words(u'english')
filtered_stopwords = [sw for sw in stopwords if sw not in ['not', 'most', 'no', 'too']]

In [30]:
def filter_out(tokens):
    return [token for token in tokens if \
            token not in stopwords and \
            token.isalpha()]

In [35]:
clean_x = ' '.join(filter_out(tokenize(x)))
clean_x[:50]

'plot two teen couples go church party drink drive '

In [42]:
f = lambda text: ' '.join(filter_out(tokenize(text)))

clean_text = []
for i, row in df.iterrows():
    text = row['text']
    clean_text.append(f(text))

In [45]:
df['clean_text'] = pd.Series(clean_text)

In [47]:
#reorder columns
df = df.loc[:, ['text', 'clean_text', 'class']]

In [50]:
df.to_csv('clean_data.csv', index=False)

# Idea 2: Add stemming

In [53]:
lemmatizer = nltk.WordNetLemmatizer()

In [66]:
f_lemmatized = lambda text: ' '.join([lemmatizer.lemmatize(token) for token in filter_out(tokenize(text))])

clean_text_lem = []
for i, row in df.iterrows():
    text = row['text']
    clean_text_lem.append(f_lemmatized(text))

In [67]:
df['clean_text_lemma'] = pd.Series(clean_text_lem)

In [68]:
df = df.loc[:, ['text', 'clean_text', 'clean_text_lemma', 'class']]

In [83]:
df.to_csv('clean_data.csv', index=False)

# Idea 3: Add Syntactic negation

In [132]:
# 'not' is in stopwords
def filter_out_neg(tokens):
    return [token for token in tokens if \
            token not in filtered_stopwords and \
            token.isalpha()]

In [140]:
cnt_negations = 0
def join_negation(tokens):
    global cnt_negations
    clean_tk = []
    skip_next = False
    for t, t_next in zip(tokens, tokens[1:]):
        if skip_next:
            skip_next = False
            continue
        if t == 'not' and t_next:
            clean_tk.append(t + "_" + t_next)
            skip_next = True
            cnt_negations += 1
        else:
            clean_tk.append(t)
    clean_tk.append(tokens[-1])
    return clean_tk

In [141]:
f_negation = lambda text: ' '.join(join_negation(
                                                filter_out_neg(
                                                tokenize(text))))

clean_text_neg = []
for i, row in df.iterrows():
    text = row['text']
    clean_text_neg.append(f_negation(text))

In [143]:
df['clean_text_neg'] = pd.Series(clean_text_neg)

In [144]:
df = df.loc[:, ['text', 'clean_text', 'clean_text_lemma', 'clean_text_neg', 'class']]

In [145]:
df.to_csv('clean_data.csv', index=False)